<a href="https://colab.research.google.com/github/Shujaat123/Ear_Biometrics/blob/main/Clean_Pytorch_Implementation_Ear_Biometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Load useful packages
!pip install wget
!pip install py7zr

In [2]:
import py7zr
from zipfile import ZipFile
from random import sample
import PIL.Image as Image
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split
from os import listdir
from os import path
import h5py
import numpy as np
import wget

In [4]:
# LOADING IITD Dataset
data_path = 'https://github.com/Shujaat123/Ear_Biometrics/blob/main/datasets/IITD_Dataset.7z?raw=true'
filename = 'IITD_Dataset.7z'
if(path.exists(filename)):
  !rm $filename
  print('existing file:', filename, ' has been deleted')
print('downloading latest version of file:', filename)
wget.download(data_path, filename)
print('DONE')

with py7zr.SevenZipFile('IITD_Dataset.7z', mode='r') as z:
    z.extractall()
!ls

# Processing IITD_dataset
src_dir = 'ear/processed/221'
images_name = listdir(src_dir)
images_name_temp = []
subjects = []
for img_ind in range(0,len(images_name)):
  if(not(images_name[img_ind]=='Thumbs.db')):
    subjects.append(int(images_name[img_ind].split('_')[0]))
    images_name_temp.append(images_name[img_ind])

images_name = images_name_temp
images_name_ord = []
subjects_ord = []

sub_ind = sorted(range(len(subjects)),key=subjects.__getitem__)
for pos, item in enumerate(sub_ind):
  images_name_ord.append(images_name[item])
  subjects_ord.append(subjects[item])

images_name = images_name_ord
subjects = subjects_ord

print(subjects)
print(images_name)

img_ind = 0
ear_images = []
sub_labels = [];
target_size = (180, 50)

for sub_ind in range(0,len(subjects)):
  img_path = src_dir+'/'+images_name[sub_ind]
  ear_img = (plt.imread(img_path))/255

  ear_img = Image.open(img_path)
  ear_img = ear_img.resize(target_size, Image.ANTIALIAS)
  ear_img = np.asarray(ear_img).astype(np.float32)/255

  ear_images.append(ear_img)
  sub_labels.append(subjects[sub_ind]-1)

ear_images = np.array(ear_images)
sub_labels = np.array(sub_labels)

print(ear_images.shape)
print(sub_labels.shape)

existing file: IITD_Dataset.7z  has been deleted
downloading latest version of file: IITD_Dataset.7z
DONE
ear  IITD_Dataset.7z  IITD_Data_Tensor.mat  IITD_Labels.mat  sample_data
[1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 51, 51, 51, 52, 52, 52, 52, 53, 53, 53, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 56, 56, 56, 56, 57, 57, 57, 57, 58, 58, 5

<ipython-input-4-887dcca04adf>:50: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  ear_img = ear_img.resize(target_size, Image.ANTIALIAS)


(793, 50, 180)
(793,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(ear_images, sub_labels, test_size=0.382093316519, random_state=42, stratify=sub_labels)
# X_train, X_test, y_train, y_test = train_test_split(ear_images, sub_labels, test_size=0.2786885245901639, random_state=42, stratify=sub_labels)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.386363636363, random_state=42, stratify=y_train)

print('Training dataset:\n',X_train.shape)
print(y_train.shape)
# print('Validation dataset:\n',X_valid.shape)
# print(y_valid.shape)
print('Test dataset:\n',X_test.shape)
print(y_test.shape)

Training dataset:
 (490, 50, 180)
(490,)
Test dataset:
 (303, 50, 180)
(303,)


# Conventional convolution

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn
import torch.nn.functional
import torch.optim
from torchvision import models #just for debugging

In [7]:
class Pytorch_BUS_Final_Model_C1(torch.nn.Module):
  #  Determine what layers and their order in CNN object
  def __init__(self, num_classes=221, num_filters=8, input_shape=(180,50,1)):
    super(Pytorch_BUS_Final_Model_C1,self).__init__()
    #self.encoder_input = input_shape[-1]
    kernel_size = 3
    # Encoder Layer1
    self.encoder_layer1_name = 'encoder_layer1'
    self.encoder_layer1_conv = torch.nn.Conv2d(1,
                                               num_filters,
                                               kernel_size,
                                               padding='same')

    self.encoder_layer1_activation = torch.nn.ReLU()
    self.encoder_layer1_pooling = torch.nn.MaxPool2d(kernel_size=(2, 2))

    # Encoder Layer2
    self.encoder_layer2_name = 'encoder_layer2'
    self.encoder_layer2_conv = torch.nn.Conv2d(num_filters,
                                               2*num_filters,
                                               kernel_size,
                                               padding='same')
    self.encoder_layer2_activation = torch.nn.ReLU()
    self.encoder_layer2_batch_norm = torch.nn.BatchNorm2d(2*num_filters,
                                                          eps = 1e-3,
                                                          momentum = 0.99)
    self.encoder_layer2_pooling = torch.nn.MaxPool2d(kernel_size=(2, 2))

    # Encoder Layer3
    self.encoder_layer3_name = 'encoder_layer3'
    self.encoder_layer3_conv = torch.nn.Conv2d(2*num_filters,
                                               4*num_filters,
                                               kernel_size,
                                               padding='same')
    self.encoder_layer3_activation = torch.nn.ReLU()
    self.encoder_layer3_pooling = torch.nn.MaxPool2d(kernel_size=(2, 2))

    # Encoder Layer4
    self.encoder_layer4_name = 'encoder_layer4'
    self.encoder_layer4_conv = torch.nn.Conv2d(4*num_filters,
                                               8*num_filters,
                                               kernel_size,
                                               padding='same')
    self.encoder_layer4_activation = torch.nn.ReLU()
    self.encoder_layer4_batch_norm = torch.nn.BatchNorm2d(8*num_filters,
                                                          eps = 1e-3,
                                                          momentum = 0.99)
    self.encoder_layer4_pooling = torch.nn.MaxPool2d(kernel_size=(2, 2))

    # Encoder Layer5
    self.encoder_layer5_name = 'encoder_layer5'
    self.encoder_layer5_conv = torch.nn.Conv2d(8*num_filters,
                                               16*num_filters,
                                               kernel_size,
                                               padding='same')

    self.encoder_layer5_activation = torch.nn.ReLU()
    self.encoder_layer5_pooling = torch.nn.MaxPool2d(kernel_size=(2, 2))

   # Encoder Layer6
    self.encoder_layer6_name = 'encoder_layer2'
    self.encoder_layer6_conv = torch.nn.Conv2d(16*num_filters,
                                               32*num_filters,
                                               kernel_size,
                                               padding='same')
    self.encoder_layer6_activation = torch.nn.ReLU()
    self.encoder_layer6_batch_norm = torch.nn.BatchNorm2d(32*num_filters,
                                                          eps = 1e-3,
                                                          momentum = 0.99)
    # Dense layer
    self.fc1_flatten = torch.nn.Flatten()
    self.fc1_linear = torch.nn.Linear(32*num_filters*(input_shape[0]//(2**5))*(input_shape[1]//(2**5)), num_classes)
    self.fc1_activation = torch.nn.Softmax()

  def forward(self,x):
    # Encoder Layer1
    out = self.encoder_layer1_conv(x)
    out = self.encoder_layer1_activation(out)
    out = self.encoder_layer1_pooling(out)

    # Encoder Layer2
    out = self.encoder_layer2_conv(out)
    out = self.encoder_layer2_activation(out)
    out = self.encoder_layer2_batch_norm(out)
    out = self.encoder_layer2_pooling(out)

    # Encoder Layer3
    out = self.encoder_layer3_conv(out)
    out = self.encoder_layer3_activation(out)
    out = self.encoder_layer3_pooling(out)

    # Encoder Layer4
    out = self.encoder_layer4_conv(out)
    out = self.encoder_layer4_activation(out)
    out = self.encoder_layer4_batch_norm(out)
    out = self.encoder_layer4_pooling(out)

    # Encoder Layer5
    out = self.encoder_layer5_conv(out)
    out = self.encoder_layer5_activation(out)
    out = self.encoder_layer5_pooling(out)

    # Encoder Layer6
    out = self.encoder_layer6_conv(out)
    out = self.encoder_layer6_activation(out)
    out = self.encoder_layer6_batch_norm(out)

    # Dense Layer
    out = self.fc1_flatten(out)
    out = self.fc1_linear(out)
    out = self.fc1_activation(out)

    return out

In [8]:
!pip install torchinfo

In [9]:
from torchinfo import summary

pytorch_model_c1 = Pytorch_BUS_Final_Model_C1()
summary(pytorch_model_c1, input_size=(1,1,180,50))

<ipython-input-7-7efb8c9265f2>:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.fc1_activation(out)


Layer (type:depth-idx)                   Output Shape              Param #
Pytorch_BUS_Final_Model_C1               [1, 221]                  --
├─Conv2d: 1-1                            [1, 8, 180, 50]           80
├─ReLU: 1-2                              [1, 8, 180, 50]           --
├─MaxPool2d: 1-3                         [1, 8, 90, 25]            --
├─Conv2d: 1-4                            [1, 16, 90, 25]           1,168
├─ReLU: 1-5                              [1, 16, 90, 25]           --
├─BatchNorm2d: 1-6                       [1, 16, 90, 25]           32
├─MaxPool2d: 1-7                         [1, 16, 45, 12]           --
├─Conv2d: 1-8                            [1, 32, 45, 12]           4,640
├─ReLU: 1-9                              [1, 32, 45, 12]           --
├─MaxPool2d: 1-10                        [1, 32, 22, 6]            --
├─Conv2d: 1-11                           [1, 64, 22, 6]            18,496
├─ReLU: 1-12                             [1, 64, 22, 6]            --
├─Bat

In [10]:
# pytorch_model_c1.eval()
# output = pytorch_model_c1(torch.Tensor(X_train[0].reshape(1,180,180,3).transpose(0,3,1,2)))
# print(output.detach().numpy())
# input_x = torch.tensor(X_train[0].reshape(1,180,50,1).transpose(0,3,1,2), device='cuda')
input_x = torch.tensor(X_train[0].reshape(1,1,180,50), device='cuda').float()
print(input_x.shape)
output = pytorch_model_c1(input_x)
print(output.shape)
print(output)

torch.Size([1, 1, 180, 50])
torch.Size([1, 221])
tensor([[0.0040, 0.0055, 0.0037, 0.0024, 0.0040, 0.0040, 0.0038, 0.0040, 0.0034,
         0.0056, 0.0039, 0.0040, 0.0024, 0.0082, 0.0123, 0.0025, 0.0066, 0.0027,
         0.0053, 0.0076, 0.0103, 0.0056, 0.0044, 0.0092, 0.0030, 0.0027, 0.0043,
         0.0038, 0.0043, 0.0058, 0.0074, 0.0065, 0.0058, 0.0043, 0.0039, 0.0048,
         0.0022, 0.0054, 0.0041, 0.0035, 0.0029, 0.0040, 0.0033, 0.0025, 0.0045,
         0.0037, 0.0066, 0.0030, 0.0092, 0.0051, 0.0045, 0.0053, 0.0141, 0.0032,
         0.0049, 0.0045, 0.0041, 0.0067, 0.0037, 0.0045, 0.0052, 0.0075, 0.0085,
         0.0034, 0.0033, 0.0064, 0.0040, 0.0020, 0.0089, 0.0040, 0.0051, 0.0059,
         0.0044, 0.0030, 0.0033, 0.0098, 0.0040, 0.0048, 0.0034, 0.0042, 0.0026,
         0.0038, 0.0030, 0.0042, 0.0032, 0.0033, 0.0089, 0.0061, 0.0041, 0.0030,
         0.0035, 0.0043, 0.0054, 0.0088, 0.0031, 0.0054, 0.0015, 0.0048, 0.0030,
         0.0028, 0.0034, 0.0077, 0.0060, 0.0052, 0.0022, 0.0

<ipython-input-7-7efb8c9265f2>:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.fc1_activation(out)


In [11]:
#data
training_loader = DataLoader(TensorDataset(torch.tensor(X_train), torch.tensor(y_train)), batch_size=10, pin_memory='True', pin_memory_device='cuda', shuffle=True)
validation_loader = DataLoader(TensorDataset(torch.tensor(X_test), torch.tensor(y_test)), batch_size=1, pin_memory='True', pin_memory_device='cuda')
#loss function
loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = torch.nn.BCELoss()
# loss_fn = torch.nn.BCEWithLogitsLoss()
# Optimizers specified in the torch.optim package
optimizer = torch.optim.Adam(pytorch_model_c1.parameters())

# # import EarlyStopping
# from pytorchtools import EarlyStopping

In [12]:

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

pytorch_model_c1 = Pytorch_BUS_Final_Model_C1().to(torch.device('cuda'))

# manaul training
def train_one_epoch():
    # training metrics
    train_loss = 0
    train_correct = 0

    # validation metrics
    valid_loss = 0
    valid_correct = 0


    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pytorch_model_c1.train(True)
    for i, data in enumerate(training_loader,0):
        # Every data instance is an input + label pair
        train_input, train_label = data
        train_input = train_input.unsqueeze(dim=1).float()
        train_label= torch.tensor(to_categorical(y=train_label, num_classes=221)).float()
        # train_label = train_label[:,None]
        if len(train_label.shape)==1:
          train_label = train_label.unsqueeze(dim=0)

        train_input = train_input.to(torch.device('cuda'))
        train_label = train_label.to(torch.device('cuda'))

        # print('train_input:',train_input.shape, 'train_label:',train_label.shape)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        train_output = pytorch_model_c1(train_input)
        # print('train_input:',train_input.shape, 'train_label:',train_label.shape, 'train_output:',train_output.shape)
        # print('train_label:',train_label)
        # print('train_output:',train_output)

        # Compute the loss and its gradients
        loss = loss_fn(train_output, train_label)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        train_loss += loss.item()
        for batch_count in range(train_output.shape[0]):
          if(torch.argmax(train_output[batch_count,:]) == torch.argmax(train_label[batch_count,:])):
            train_correct += 1

    # print('training epoch complete')
    # Here, we use enumerate(validation_loader) instead of
    # iter(validation_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pytorch_model_c1.train(False)
    for i, data in enumerate(validation_loader,0):
        # Every data instance is an input + label pair
        valid_input, valid_label = data

        valid_input = valid_input.unsqueeze(dim=1).float()
        valid_label= torch.tensor(to_categorical(y=valid_label, num_classes=221)).float()
        if len(valid_label.shape)==1:
          valid_label = valid_label.unsqueeze(dim=0)

        valid_input = valid_input.to(torch.device('cuda'))
        valid_label = valid_label.to(torch.device('cuda'))

        # Make predictions for this batch
        valid_output = pytorch_model_c1(valid_input)

        # print('valid_input:',valid_input.shape, 'valid_label:',valid_label.shape, 'valid_output:',valid_output.shape)

        # Gather data and report
        valid_loss += loss_fn(valid_output, valid_label).item()
        for batch_count in range(valid_output.shape[0]):
          if(torch.argmax(valid_output[batch_count,:]) == torch.argmax(valid_label[batch_count,:])):
            valid_correct += 1

    print(f"Training: \n Training Accuracy: {100*train_correct/len(training_loader.dataset)}%, Average Training Loss: {train_loss/len(training_loader)}")

    print(f"Validation: \n Validation Accuracy: {100*valid_correct/len(validation_loader.dataset)}%, Average Validation Loss: {valid_loss/len(validation_loader)}")

    return train_loss, valid_loss


In [13]:
# Initializing in a separate cell so we can easily add more epochs to the same run
epoch_number = 0
EPOCHS = 100
optimizer = torch.optim.Adam(pytorch_model_c1.parameters(), lr=1e-4)
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    train_loss, valid_loss = train_one_epoch()
    epoch_number += 1

EPOCH 1:


<ipython-input-7-7efb8c9265f2>:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.fc1_activation(out)


Training: 
 Training Accuracy: 1.2244897959183674%, Average Training Loss: 5.397212077160271
Validation: 
 Validation Accuracy: 3.3003300330033003%, Average Validation Loss: 5.390010459194876
EPOCH 2:
Training: 
 Training Accuracy: 5.510204081632653%, Average Training Loss: 5.380439339851846
Validation: 
 Validation Accuracy: 5.6105610561056105%, Average Validation Loss: 5.366038734763369
EPOCH 3:
Training: 
 Training Accuracy: 9.183673469387756%, Average Training Loss: 5.341619666741819
Validation: 
 Validation Accuracy: 10.231023102310232%, Average Validation Loss: 5.32459035722336
EPOCH 4:
Training: 
 Training Accuracy: 16.53061224489796%, Average Training Loss: 5.29768399803006
Validation: 
 Validation Accuracy: 15.841584158415841%, Average Validation Loss: 5.285597738259696
EPOCH 5:
Training: 
 Training Accuracy: 27.551020408163264%, Average Training Loss: 5.234065513221585
Validation: 
 Validation Accuracy: 27.062706270627064%, Average Validation Loss: 5.22485352664104
EPOCH 6:
T